In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.2/339.2 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('/content/drive/MyDrive/travel_advisor_Paris_2024/.env')) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [4]:
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

In [5]:
from crewai import Agent, Task, Crew

In [6]:
local_guide = Agent(
    role="Paris tour guide",
    goal="Recommend local attractions including 'hidden gems' and local experiences to a visitor according to"
          "their {interests}, avoid overcrowded tourist areas",
    backstory = "You have an extensive knowledge of Paris attractions and can give visitors"
                "detailed recommendations of places to visit taking their {interests} into consideration. Your recommendations always"
                "include lesser known attractions ('hidden gems') that allow the visitors"
                "to avoid crowded toursit areas and experience the authentic side of Paris.",
    allow_delegation=False,
	  verbose=True
)

In [10]:
travel_planner = Agent(
        role="Amazing travel planner",
        goal="Create the most amazing travel itineraries for travellers to Paris for {days} days",
        backstory="You are a specialist in travel planning with decades of experience."
                   "You provide a detailed travel itinerary for number of {days} days,"
                   "that includes the recommendations of the Paris tour guide",
        allow_delegation=False,
        verbose=True)

In [7]:
recommend = Task(
    description=(
        "1. Recommend local attractions for a visitor according to their"
            "{interests}. Prioritize lesser known attractions that are away from tourist crowds.\n"
        "2. Recommend authentic Parisian experiences according to "
            "visitor's {interests}.\n"
        "3. Recommend authentic local restaurants.\n "
    ),
    expected_output="A list of items, where each item includes the name of a place in Paris, location of the place,"
                    "and a short description of why this place is worth visiting",
    agent=local_guide,
)

In [11]:
plan = Task(
    description = (
        "1. Expand the recommendations of the Paris tour guide into a a full {days}-day travel"
            "itinerary with detailed per-day plans, including places to eat.\n"
        "2. This itinerary should cover all aspects of the trip, from arrival to departure,"
            "integrating the Paris tour guide information with practical travel logistics."
        "3. You MUST suggest actual places to visit and actual restaurants to go to."
    ),
    expected_output = "Your final answer MUST be a complete expanded travel plan,"
                      "formatted as markdown, encompassing a daily schedule,"
                      "ensuring THE BEST TRIP EVER. Be specific and give a reason why you picked"
                      "each place, what make them special!",
    agent=travel_planner,
)

In [12]:
crew = Crew(
    agents=[local_guide, travel_planner],
    tasks=[recommend, plan],
    verbose=2
)

In [13]:
result = crew.kickoff(inputs={"interests": "Art", "days": 3})

 [DEBUG]: == Working Agent: Paris tour guide
 [INFO]: == Starting Task: 1. Recommend local attractions for a visitor according to theirArt. Prioritize lesser known attractions that are away from tourist crowds.
2. Recommend authentic Parisian experiences according to visitor's Art.
3. Recommend authentic local restaurants.
 


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
1. Place: Musée de la Vie Romantique
   Location: 16 Rue Chaptal, 75009 Paris
   Description: This hidden gem museum is dedicated to the Romantic period in Paris, offering a peaceful and intimate atmosphere for art enthusiasts to explore paintings, sculptures, and personal belongings of famous artists from the 19th century.

2. Place: Atelier des Lumières
   Location: 38 Rue Saint-Maur, 75011 Paris
   Description: Experience art in a unique way at this digital art center where immersive exhibitions project famous artworks onto the walls, floors, and ceilings, creating a mesmeri